<a href="https://colab.research.google.com/github/anitayadav3/EmotionRecognitionInConversation/blob/master/WordEmbedding_Glove.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [10]:
import pickle
import pandas as pd
import numpy as np
import keras
import itertools
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Embedding, Bidirectional
from keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical  

In [5]:
glove_6B_100d_file_path_name = "/content/gdrive/My Drive/iemocap/glove.6B.100d.txt"

embeddings_index = dict()

f = open(glove_6B_100d_file_path_name)

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
    
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [7]:
all_glove_words = list(embeddings_index.keys())
vocabulary_size = len(all_glove_words)
tokenizer = Tokenizer() #num_words= vocabulary_size
tokenizer.fit_on_texts(all_glove_words)

In [22]:
print(len(all_glove_words))

400000


In [8]:
embedding_matrix = np.zeros((vocabulary_size, 100)) 
for word, index in tokenizer.word_index.items():
    if index > vocabulary_size - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

In [11]:
with open('/content/gdrive/My Drive/iemocap/train/sentences.pkl', 'rb') as f:
    data = pickle.load(f)
with open('/content/gdrive/My Drive/iemocap/train/labels.pkl', 'rb') as f:
    labels = pickle.load(f)
with open('/content/gdrive/My Drive/iemocap/test/sentences.pkl', 'rb') as f:
    test_data = pickle.load(f)
with open('/content/gdrive/My Drive/iemocap/test/labels.pkl', 'rb') as f:
    test_labels = pickle.load(f)

In [12]:
def preprocessing(data,labels):
  processed_data=[]
  processed_label=[]
  for i in range(0,len(data)):
    for j in range(0,len(data[i])):
      intermediate_data=[]
      intermediate_label=[]
      for k in range(0,len(data[i][j])):
        text=data[i][j][k]
        if text != '<eos>'and text!='<pad>':
          intermediate_data.append(text)
      processed_data.append(intermediate_data)
  for i in labels:
    for j in i:
      processed_label.append(j)
  return processed_data,processed_label

In [13]:
processed_data,processed_label = preprocessing(data,labels)
test_processed_data,test_processed_label = preprocessing(test_data,test_labels)

In [20]:
def Tokenize(processed_data,processed_label):
  X = tokenizer.texts_to_sequences(processed_data)
  X = pad_sequences(X)
  Y = to_categorical(processed_label, num_classes=6)
  return X,Y

In [21]:
X,Y = Tokenize(processed_data, processed_label)
test_X , test_Y = Tokenize(test_processed_data,test_processed_label)

In [24]:
lstm_out = 200
embed_dim=100
model = Sequential()
model.add(Embedding(len(all_glove_words), embed_dim,input_length = X.shape[1],weights=[embedding_matrix],trainable=False))
model.add(Bidirectional(LSTM(200)))
model.add(Dense(6,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 29, 100)           40000000  
_________________________________________________________________
bidirectional (Bidirectional (None, 400)               481600    
_________________________________________________________________
dense (Dense)                (None, 6)                 2406      
Total params: 40,484,006
Trainable params: 484,006
Non-trainable params: 40,000,000
_________________________________________________________________
None


In [25]:
model.fit(X, Y, epochs = 10,  verbose = 1)
score,acc=model.evaluate(test_X,test_Y,verbose=2)
print("Accuracy : "+ str(acc))

Epoch 1/10
147/147 [==============================] - 4s 24ms/step - loss: 1.6875 - accuracy: 0.2930
Epoch 2/10
147/147 [==============================] - 3s 24ms/step - loss: 1.5872 - accuracy: 0.3507
Epoch 3/10
147/147 [==============================] - 4s 24ms/step - loss: 1.4956 - accuracy: 0.4075
Epoch 4/10
147/147 [==============================] - 3s 24ms/step - loss: 1.4131 - accuracy: 0.4561
Epoch 5/10
147/147 [==============================] - 3s 24ms/step - loss: 1.3309 - accuracy: 0.4897
Epoch 6/10
147/147 [==============================] - 4s 24ms/step - loss: 1.2369 - accuracy: 0.5305
Epoch 7/10
147/147 [==============================] - 3s 24ms/step - loss: 1.1355 - accuracy: 0.5667
Epoch 8/10
147/147 [==============================] - 4s 24ms/step - loss: 1.0335 - accuracy: 0.6093
Epoch 9/10
147/147 [==============================] - 4s 24ms/step - loss: 0.9160 - accuracy: 0.6593
Epoch 10/10
147/147 [==============================] - 4s 24ms/step - loss: 0.8142 - accura